In [1]:
import pandas as pd
import psycopg2
import numpy as np



from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service



from tqdm import tqdm
tqdm.pandas()
import time



from bs4 import BeautifulSoup
import re

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders


In [2]:
#scraping only brand and categories, rest of tha data is available in serp

In [3]:
s = Service("../chromedriver_win32/chromedriver.exe")

In [4]:
chrome_opt=Options()
chrome_opt.add_argument("--incognito")
driver = webdriver.Chrome(service=s,options=chrome_opt)
capabilities = driver.capabilities
chrome_driver_version = capabilities['chrome']['chromedriverVersion']

# Print the ChromeDriver version
print("ChromeDriver version:", chrome_driver_version)


ChromeDriver version: 114.0.5735.90 (386bc09e8f4f2e025eddae123f36f6263096ae49-refs/branch-heads/5735@{#1052})


In [5]:
class postgres_conn:
    def getConn(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='postgres_007',
                                          host="1.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error
    def getConn2(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='reporting_007',
                                          host="2.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error


    def close_connection(self, cursor, connection):
        # closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
            
pg = postgres_conn()
conn = pg.getConn()
conn2 = pg.getConn2()

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '1.pgsql.db.1digitalstack.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '2.pgsql.db.1digitalstack.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [31]:
job_id=input("Enter keystone ids with comma.").split(",")
days=input(" Enter no of days for which data needs brand mapping.\n")
job_id.append("")
tuple(job_id)

Enter keystone ids with comma.64d1d10b59aa371d9ed1a176
 Enter no of days for which data needs brand mapping.
4


('64d1d10b59aa371d9ed1a176', '')

In [32]:
query = f"""
select brand,title,sku,job_name,channel_id
from ready.serp_details smd join ready.serp_mongo_details m on m.serp_mongo_id=smd.serp_mongo_id
where smd.crawl_date > now() - interval '{days} day' and keystone_job_id in {tuple(job_id)}
"""
serp_details = pd.read_sql_query(query, conn[1])

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_149436\2162843895.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  serp_details = pd.read_sql_query(query, conn[1])


In [33]:
serp_details

,brand,title,sku,job_name,channel_id
0,None,Immersion Hand Blender 3-in-1 Blender Bar,MLM-1916833531,Mercadolibre r.no.565 Serp_job,46
1,None,Immersion Blender Hand Mixer,MLM-2209098724,Mercadolibre r.no.565 Serp_job,46
2,None,Immersion Hand Blender 12 Speeds,MLM-1915104199,Mercadolibre r.no.565 Serp_job,46
3,None,"1-Touch Control Hand Blender, 150 Watts,",MLM-2331361108,Mercadolibre r.no.565 Serp_job,46
4,None,5 In1 800w Immersion Hand Blender,MLM-2324849644,Mercadolibre r.no.565 Serp_job,46
...,...,...,...,...,...
13712,None,Turmix TXBL01 2 L blender with polycarbonate g...,MLM13170712,Mercadolibre r.no.565 Serp_job,46
13713,None,Koblenz Klm 9510 Professional Heavy Duty Blend...,MLM-1385254324,Mercadolibre r.no.565 Serp_job,46
13714,None,Industrial blender International LI-17A 17 L s...,MLM15900310,Mercadolibre r.no.565 Serp_job,46
13715,None,Industrial blender Mixer 03-S 3 L polished sta...,MLM18375485,Mercadolibre r.no.565 Serp_job,46


In [34]:
df=serp_details.copy()

In [35]:
df.drop_duplicates(subset='sku', inplace=True)
df.dropna(subset='sku', inplace=True)
df.reset_index(drop=True, inplace=True)
df

,brand,title,sku,job_name,channel_id
0,None,Immersion Hand Blender 3-in-1 Blender Bar,MLM-1916833531,Mercadolibre r.no.565 Serp_job,46
1,None,Immersion Blender Hand Mixer,MLM-2209098724,Mercadolibre r.no.565 Serp_job,46
2,None,Immersion Hand Blender 12 Speeds,MLM-1915104199,Mercadolibre r.no.565 Serp_job,46
3,None,"1-Touch Control Hand Blender, 150 Watts,",MLM-2331361108,Mercadolibre r.no.565 Serp_job,46
4,None,5 In1 800w Immersion Hand Blender,MLM-2324849644,Mercadolibre r.no.565 Serp_job,46
...,...,...,...,...,...
2297,None,T-fal blender + coffee maker + frying pan,MLM-1889177199,Mercadolibre r.no.565 Serp_job,46
2298,None,Holstein Housewares Oriflame 3-in-1 Immersion ...,MLM24750875,Mercadolibre r.no.565 Serp_job,46
2299,None,Holstein 3in1 Immersion Blender Style AND Ease!,MLM-1916024585,Mercadolibre r.no.565 Serp_job,46
2300,None,Seville| Blender With Digital Panel | 550W | 5...,MLM23371259,Mercadolibre r.no.565 Serp_job,46


In [36]:
asinlist = list(df['sku'])
len(asinlist)

2302

In [37]:
query_2 = f""" 
Select channel_sku_id,country_code, brand_name, categories,category_id
from entity.product_master
where channel_sku_id in {tuple(asinlist)} 
"""

In [38]:
pm_data = pd.read_sql_query(query_2, conn[1])
pm_data

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_149436\3768858016.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pm_data = pd.read_sql_query(query_2, conn[1])


,channel_sku_id,country_code,brand_name,categories,category_id
0,MLM-1301021404,MX,Oster,"['...', '...', 'Para Cocina', 'Repuestos y Acc...",None
1,MLM-1303019390,MX,Oster,"['...', '...', 'Para Cocina', 'Repuestos y Acc...",None
2,MLM-1303945854,MX,Ilko,"['Hogar, Muebles y Jardín', 'Cocina', 'Cocción...",None
3,MLM-1303976050,MX,Oster,"['Electrodomésticos', 'Pequeños Electrodomésti...",None
4,MLM-1304001270,MX,Nutribullet,"['...', '...', 'Para Cocina', 'Repuestos y Acc...",None
...,...,...,...,...,...
2290,MLM9872359,MX,Ninja,"['Electrodomésticos', 'Pequeños Electrodomésti...",None
2291,MLM9922827,MX,Vitamix,"['Electrodomésticos', 'Pequeños Electrodomésti...",None
2292,MLM9922830,MX,Vitamix,"['Electrodomésticos', 'Pequeños Electrodomésti...",None
2293,MLM9994415,MX,Vitamix,"['Electrodomésticos', 'Pequeños Electrodomésti...",None


In [39]:
mapped_df=pd.merge(df,pm_data, how='left', left_on=['sku'],right_on=['channel_sku_id'])

In [40]:
mapped_df

,brand,title,sku,job_name,channel_id,channel_sku_id,country_code,brand_name,categories,category_id
0,None,Immersion Hand Blender 3-in-1 Blender Bar,MLM-1916833531,Mercadolibre r.no.565 Serp_job,46,MLM-1916833531,MX,Original,"['Electrodomésticos', 'Pequeños Electrodomésti...",None
1,None,Immersion Blender Hand Mixer,MLM-2209098724,Mercadolibre r.no.565 Serp_job,46,MLM-2209098724,MX,Original,"['Electrodomésticos', 'Pequeños Electrodomésti...",None
2,None,Immersion Hand Blender 12 Speeds,MLM-1915104199,Mercadolibre r.no.565 Serp_job,46,MLM-1915104199,MX,Original,"['Electrodomésticos', 'Pequeños Electrodomésti...",None
3,None,"1-Touch Control Hand Blender, 150 Watts,",MLM-2331361108,Mercadolibre r.no.565 Serp_job,46,MLM-2331361108,MX,Original,"['Electrodomésticos', 'Pequeños Electrodomésti...",None
4,None,5 In1 800w Immersion Hand Blender,MLM-2324849644,Mercadolibre r.no.565 Serp_job,46,MLM-2324849644,MX,Original,"['Electrodomésticos', 'Pequeños Electrodomésti...",None
...,...,...,...,...,...,...,...,...,...,...
2297,None,T-fal blender + coffee maker + frying pan,MLM-1889177199,Mercadolibre r.no.565 Serp_job,46,MLM-1889177199,MX,T-Fal,"['Electrodomésticos', 'Pequeños Electrodomésti...",None
2298,None,Holstein Housewares Oriflame 3-in-1 Immersion ...,MLM24750875,Mercadolibre r.no.565 Serp_job,46,MLM24750875,MX,Holstein,"['Electrodomésticos', 'Pequeños Electrodomésti...",None
2299,None,Holstein 3in1 Immersion Blender Style AND Ease!,MLM-1916024585,Mercadolibre r.no.565 Serp_job,46,MLM-1916024585,MX,Holstein,"['Electrodomésticos', 'Pequeños Electrodomésti...",None
2300,None,Seville| Blender With Digital Panel | 550W | 5...,MLM23371259,Mercadolibre r.no.565 Serp_job,46,MLM23371259,MX,Taurus,"['Industrias y Oficinas', 'Gastronomía y Hotel...",None


In [41]:
df2=mapped_df[mapped_df['categories'].isnull()][['channel_id','sku','title','brand_name','categories']]
#only those rows where categories couloumn is null
df2.reset_index(drop=True,inplace=True)

In [42]:
df2

,channel_id,sku,title,brand_name,categories
0,46,MLM-1429201335,"Vitamix, Red 7500 Blender, Professional Grade,...",NaN,NaN
1,46,MLM-1421714052,"Vitamix Professional Series Blender, Blender, ...",NaN,NaN
2,46,MLM-679760324,"Oster Shedmonster Shedding Tool, Fine Or Ch",NaN,NaN
3,46,MLM-1851731967,Frying Pan Royal Prestige 26cm Original New Gi...,NaN,NaN
4,46,MLM-1527945298,Original Royal Prestige Valve,NaN,NaN
5,46,MLM-1700881852,Original Universal Royal Prestige Valve,NaN,NaN
6,46,MLM-2385624292,Frying Pan And Pot Royal Prestige,NaN,NaN


In [28]:
asinlist = list(df2['sku'])
len(asinlist)

6

In [46]:
batches=int(len(asinlist)/50 )+1
batches

47

In [43]:

for batch in tqdm(range(0,len(asinlist))):
    print(batch)
    for val in (range(batch*50,(batch*50)+50)):

        i=asinlist[val]
        print(batch,i)
        if "-" in i:
            #Url== "https://produto.mercadolibre.com.br/{}".format(i)
            Url = "https://articulo.mercadolibre.com.mx/{}".format(i)
        else:
            #Url="https://www.mercadolivre.com.br//p/{}".format(i)
            Url="https://www.mercadolibre.com.mx//p/{}".format(i)
        
        
        #Url = "https://produto.mercadolibre.com.br/{}".format(i)
        #Url = "https://articulo.mercadolibre.com.mx/{}".format(i)
        print(Url)
        value=''
        #s= get_session(proxies)
        r= requests.get(Url,headers=headers)

        soup=BeautifulSoup(r.content,'html.parser')

        try:
            main=soup.find('div',class_="ui-pdp-specs__table")
            labels=[labels.text for labels in main.find_all('th')]
            try:
                index=labels.index('Marca')
                value=soup.find_all('td')[index].text
                print(value)
            except:
                value= 'not available'
                print(value)
        except:
            pass

        try:
            main=soup.find('div',class_="ui-vpp-striped-specs")
            labels=[labels.text for labels in main.find_all('th')]
            try:
                index=labels.index('Marca')
                value=soup.find_all('td')[index].text
                print(value)
            except:
                value= 'not available'
                print(value)
        except:
            pass
        
        try:
            category=soup.find('div',class_="ui-pdp-breadcrumb")
            cats=[labels.text for labels in category.find_all('li')]
            print(cats)
        except:
            cats=[]
            
        try:
            sales=soup.find('div',class_='ui-pdp-header')
            d=sales.find('span',class_='ui-pdp-subtitle')
            sold=d.text
            print(sold)
        except:
            sold='na'
            print(sold)




        df1={'channel_sku_id': i, 'brand_name': value, 'categories': cats, 'sold':sold}
        df=df.append(df1, ignore_index = True) 
    time.sleep(300)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

************************
0
https://articulo.mercadolibre.com.mx/MLM-1429201335


  2%|█▋                                                                                 | 1/50 [00:02<02:10,  2.66s/it]

001970
['Hogar, Muebles y Jardín', 'Adornos y Decoración del Hogar', 'Otros']
************************
1
https://articulo.mercadolibre.com.mx/MLM-1421714052


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_149436\3509711186.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['brand_name'][i]=value
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_149436\3509711186.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['categories'][i]=cats
  4%|███▎                                                                               | 2/50 [00:04<01:50,  2.30s/it]

...

 12%|█████████▉                                                                         | 6/50 [00:13<01:26,  1.98s/it]

['Hogar, Muebles y Jardín', 'Cocina', 'Otros']
************************
6
https://articulo.mercadolibre.com.mx/MLM-2385624292


 14%|███████████▌                                                                       | 7/50 [00:16<01:41,  2.36s/it]

['Hogar, Muebles y Jardín', 'Cocina', 'Cocción y Horneado', 'Baterías, Ollas y Sartenes', 'Baterías Completas']


KeyError: 7

In [44]:
final1=pd.merge(df,df2,on='sku', how= 'right')
#final=final1[['sku','title','Asin','Brand','Category']]

In [45]:
final1

,brand,title_x,sku,job_name,channel_id_x,channel_id_y,title_y,brand_name,categories
0,None,"Vitamix, Red 7500 Blender, Professional Grade,...",MLM-1429201335,Mercadolibre r.no.565 Serp_job,46,46,"Vitamix, Red 7500 Blender, Professional Grade,...",001970,"[Hogar, Muebles y Jardín, Adornos y Decoración..."
1,None,"Vitamix Professional Series Blender, Blender, ...",MLM-1421714052,Mercadolibre r.no.565 Serp_job,46,46,"Vitamix Professional Series Blender, Blender, ...",Vitamix,"[Industrias y Oficinas, Otros]"
2,None,"Oster Shedmonster Shedding Tool, Fine Or Ch",MLM-679760324,Mercadolibre r.no.565 Serp_job,46,46,"Oster Shedmonster Shedding Tool, Fine Or Ch",NaN,"[Electrodomésticos, Pequeños Electrodomésticos..."
3,None,Frying Pan Royal Prestige 26cm Original New Gi...,MLM-1851731967,Mercadolibre r.no.565 Serp_job,46,46,Frying Pan Royal Prestige 26cm Original New Gi...,NaN,"[Hogar, Muebles y Jardín, Cocina, Cocción y Ho..."
4,None,Original Royal Prestige Valve,MLM-1527945298,Mercadolibre r.no.565 Serp_job,46,46,Original Royal Prestige Valve,NaN,"[Hogar, Muebles y Jardín, Cocina, Otros]"
5,None,Original Universal Royal Prestige Valve,MLM-1700881852,Mercadolibre r.no.565 Serp_job,46,46,Original Universal Royal Prestige Valve,NaN,"[Hogar, Muebles y Jardín, Cocina, Otros]"
6,None,Frying Pan And Pot Royal Prestige,MLM-2385624292,Mercadolibre r.no.565 Serp_job,46,46,Frying Pan And Pot Royal Prestige,NaN,"[Hogar, Muebles y Jardín, Cocina, Cocción y Ho..."
